In [1]:
# Importing packages
import numpy as np
from sympy import solve, symbols, Eq
import random 
from tqdm import tqdm
import itertools

# Importing Pauli matrices
from pauli_matrices.pauli_matrices import I, X, Y, Z

# Importing openfermion
from openfermion.ops import FermionOperator
from openfermion.transforms import jordan_wigner, bravyi_kitaev

from pauli_matrices.pauli_chain import get_Pauli_chain
from hamiltonian.hamiltonian import get_hamiltonian


In [2]:
def get_Bell_term(measurement_choices, measurement_indices):
            
    # Obtaining Pauli chain
    Pauli_chain = get_Pauli_chain(measurement_choices)

    # Obtaining the variable indices
    lower_string = ''

    for j in measurement_indices:
        lower_string += str(j)

    # Finilizing the variable
    var_name = 'c_{' + lower_string + '}'
    var = symbols(var_name)

    # Returning the Bell operator term
    return var * Pauli_chain, var_name, var



In [47]:
# Defining the number of qubits
N = 2

# Defining the Hamiltonian
H = np.kron(Z, Z) + np.kron(X, X)

# Defining the measurement choices for Alice and Bob
A_0 = X
A_1 = Z
B_0 = 1/np.sqrt(2)*(X+Z)
B_1 = 1/np.sqrt(2)*(X-Z)

# Defining expansion coefficients C_ij
coeffs = []
for i in range(N+1):
    for j in range(N+1):
        coeffs.append( symbols(('C_'+str(i)+str(j))) )

# Calculating the eigenvalues and eigenstates
eig_vals, eig_vecs = np.linalg.eigh(H)

# extracting the ground state energy and the respective eigenstate
E_G = eig_vals[0]
psi_G = eig_vecs[:,0]
psi_G_dagger = np.conjugate(np.transpose(psi_G))

# # Adding the ground state energy to the Bell operator 
# # Bell_operator += -E_G * np.kron(I,I)



In [4]:
# Consrtucting a measurement matrix
# The Identities correspond to the case in which we do not perform a measurement on a specific qubit
M = np.array([
    [I, A_0, A_1],
    [I, B_0, B_1]
])


# Initializing the Bell operator, the Bell inequality and a string containing the variables
B = 0
var = []
var_dict = {}

# Creating indices for all measurements
temp_indices = list(itertools.product([i for i in range(N+1)], repeat=(N)))

# Checking for one and two body terms requirement
indices = []
one_and_two_body = False

if one_and_two_body:
    for j in temp_indices:
        if np.sum([j[i] == 0 for i in range(len(j))]) >= 2:
            indices.append(j)
else:
    indices = temp_indices.copy()

# Calculating the Bell operator
for measurement_indices in tqdm(indices):

    # Constructing the measurement choices
    measurement_choices = [M[p, measurement_indices[p]] for p in range(N)]

    # Calculating the Bell terms and the corresponding variable
    B_temp, var_name, var_temp = get_Bell_term(measurement_choices, measurement_indices)
    
    # Adding and storing the values
    B += B_temp
    var.append(var_temp)
    var_dict[var_name] = var_temp
# 

100%|██████████| 9/9 [00:00<00:00, 81.08it/s]


In [5]:
# Obtaining the sytsem of equations (this takes ages)
pauli_matrices = [I, X, Y, Z]

# Initializing the equations list
eqs = []

# Calculating new set of indices
indices = list(itertools.product([i for i in range(len(pauli_matrices))], repeat=N))

# Looping over the equations
for projector_indices in tqdm(indices):

    # Constructing the measurement choices
    measurement_choices = [pauli_matrices[p] for p in projector_indices]
    Projector = get_Pauli_chain(measurement_choices)

    # calculating system of equations
    eqs.append( 
        Eq( 
            np.trace( np.matmul(B, Projector)).simplify(), 
            np.trace( np.matmul(H, Projector))
        ) 
    )
   

100%|██████████| 16/16 [00:00<00:00, 35.95it/s]


In [6]:
# Solving the system of equations
ans = solve(eqs, var)

# Updating the dictionary
for key in ans.keys():
    var_dict[str(key)] = ans[key]

In [34]:
# Initializing E_0 and the numbre of measurements m per party
m = 2      # Number of possible measurements
E_i = 0

# Initializing the matrix M
# 0'th row does not correspond to a measurement
M_c = np.ones((N+1, N))   # DOes this need to be initialized in a random manner?

# Initializing temporary indices
temp_indices = list(itertools.product([i for i in range(m+1)], repeat=N))

# Calculating the possible outcomes for the i'th row of the matrix M
possible_configurations = list(itertools.product([1, -1], repeat=m))

# Calculating the i'th indices
for idx in range(N):

    # This needs to be done for each iteration of idx, the above does not have to be done each iteration
    indices = []

    # First condition ensures that we do not have any double counting
    # Second condition ensures that we do not take a constant non measurable value into account
    # Last condition makes it so we do not capture any measurements from other observables
    for j in temp_indices:

        if np.sum(j[:idx]) == 0 and np.sum(j) != 0 and j[idx] != 0: 
            indices.append([*[0] * (idx), *j[idx:]])

    # Updating correlation matrix and setting a very high value for h_i such that we always start with a lowest configuration after one iteration
    M_c_temp = M_c.copy()
    h_i = 1e6

    # Looping over the different configurations
    for conf in possible_configurations:

        # Updating the correlation matrix
        M_c_temp[1:, idx] = conf

        # Extracting the coefficients and the matrix products
        coeffs = []
        M_elements = []

        for single_idx in indices:

            # Initializing coefficients and matrix products
            name = ''
            M_prod = 1

            # Looping over the m measurements
            for j in range(m):
                name += str(single_idx[j])
                M_prod *= M_c_temp[single_idx[j], j]

            # Storing values
            coeffs.append(var_dict['c_{'+name+'}'])
            M_elements.append(M_prod)

        # Calculating h_i
        h_i_temp = np.sum([coeffs[j]*M_elements[j] for j in range(len(indices))])

        # Checking if the new h_i is smaller than the old one, if yes, then we adapt the new value
        if h_i_temp < h_i:
            M_c = M_c_temp.copy()
            h_i = h_i_temp

    # Adding the last configuration of h_i to E_i
    E_i += h_i

# Obtaining the classical bound
beta_C = -E_i


In [44]:
# Obtaining all possible indices and configurations
indices = list(itertools.product([i for i in range(N+1)], repeat=N))
possible_configurations = list(itertools.product([1, -1], repeat=N**2))

# Initializing Bell inequality
I = 1e6

# Initializing the correlation matrix
M_c = np.ones((N+1, N))

for conf in possible_configurations:
    # Extracting the coefficients and the matrix products
    coeffs = []
    M_elements = []

    # Updating correlation matrix
    M_c[1:, :] = np.reshape(conf, (N, N))

    for idx in indices:
        # Initializing coefficients and matrix products
        name = ''
        M_prod = 1

        # Looping over the m measurements
        for j in range(m):
            name += str(idx[j])
            M_prod *= M_c[idx[j], j]

        # Storing values
        coeffs.append(var_dict['c_{'+name+'}'])
        M_elements.append(M_prod)
    
    # Calculating the Bell inequality
    I_new = np.sum([coeffs[j]*M_elements[j] for j in range(len(indices))])

    # Checking if we have a new minimum
    if I_new < I:
        I = I_new

beta_C = -I


In [49]:
# Obtaining the Bell operator 
for i in range(2**N):
    for j in range(2**N):
        for variable in var:
            B[i,j] = B[i,j].subs(variable, ans[variable])

# Calculting the quantum value
beta_Q = np.matmul(psi_G_dagger, np.matmul(B, psi_G))

In [51]:
print(r'beta_Q = %.3f and beta_C = %.3f' %(beta_Q, beta_C))

beta_Q = -2.000 and beta_C = 1.414
